<table><tr>
<td> <img src="https://upload.wikimedia.org/wikipedia/fr/thumb/e/e5/Logo_%C3%A9cole_des_ponts_paristech.svg/676px-Logo_%C3%A9cole_des_ponts_paristech.svg.png" width="200"  height="200" hspace="200"/> </td>
<td> <img src="https://pbs.twimg.com/profile_images/1156541928193896448/5ihYIbCQ_200x200.png" width="200" height="200" /> </td>
</tr></table>

<br/>

<h1><center>Advanced Modelling</center></h1>



<font size="3">This session is divided into **2** parts:
- **Multi Layer Perceptron**
- **--jnjd**
>  * 1-Hyperparameters optimization
>  * 2-Features selection

In each of these parts, some **guidelines** and **hints** are given for each task. 
Do not hesitate to check the links to documentation to understand the functions you use. 
    
The goal of this session is to **select a model** that you will use as your best candidate and optimize it to get the best out of it.
</font>

## Load data

In [16]:
import pandas as pd

def load_dataset(path):
    print(f"loading raw data..")
    data = pd.read_csv(path)
    data.drop(['title'], axis = 1, inplace = True)
    return data

def clean_data(data, drop_2020=True):
    print(f"cleaning data..")
    data = data.dropna()
    if drop_2020:
        data = data.query("year != 2020")
    data = data.sort_values(by='release_date')
    data.release_date = pd.to_datetime(data.release_date)
    data.index = data.release_date
    data = data.drop(columns = ['index', 'release_date', 'year'], errors='ignore')
    return data

def train_test_split_by_date(df: pd.DataFrame, split_date_val: str, split_date_test: str):
    """Split dataset according to a split date in format "YYYY-MM-DD"
    - train: [:split_date_1[
    - validation: [split_date_1: split_date_2[
    - test: [split_date_2:[
    """
    train = df.loc[:split_date_val].copy()
    validation = df.loc[split_date_val:split_date_test].copy()
    test = df.loc[split_date_test:].copy()
    return train, validation, test

def get_x_y(dataset):
    target = dataset.sales
    target = target.astype(float)
    features = dataset.drop(columns = ['sales'], errors='ignore')
    return features, target

In [17]:
path = '/Users/yaguethiam/Ponts/data_prepared_session4.csv'
raw_data = load_dataset(path)

loading raw data..


In [261]:
data = clean_data(raw_data, drop_2020=False)
train_data, validation_data, test_data = train_test_split_by_date(data,
                                                                  '2018-01-01',
                                                                  '2020-01-01')
train_x, train_y = get_x_y(train_data)
validation_x, validation_y = get_x_y(validation_data)
test_x, test_y = get_x_y(test_data)

cleaning data..


## Linear model

In [262]:
import torch
def normalize(x, m, s): return (x-m)/s
def mse(output, targ): return (output.squeeze(-1) - targ).pow(2).mean()

In [263]:
train_mean,train_std = train_x.mean(),train_x.std()

In [264]:
x_train = normalize(train_x, train_mean, train_std)
# NB: Use training, not validation mean for validation set
x_valid = normalize(validation_x, train_mean, train_std)

In [265]:
x_train  = torch.tensor(x_train.values)
x_valid  = torch.tensor(x_valid.values)

In [266]:
x_train = x_train.float()

y_train = torch.tensor(train_y.values).log().unsqueeze(1).float()

In [267]:
m = 47
nh = 50
n=5958
bs=1000                  # batch size

xb = x_train[0:bs]     # a mini-batch from x
preds = model(xb)      # predictions
preds[0], preds.shape
yb= y_train[0:bs]  

In [268]:
lr = 0.1   # learning rate
epochs = 10 # how many epochs to train for

In [269]:
loss_func = nn.L1Loss()

In [270]:
def fit():
    for epoch in range(epochs):
        for i in range((n-1)//bs + 1):
            start_i = i*bs
            end_i = start_i+bs
            xb = x_train[start_i:end_i]
            yb = y_train[start_i:end_i]
            loss = loss_func(model(xb), yb)
            print(loss)
            loss.backward()
            with torch.no_grad():
                for p in model.parameters(): p -= p.grad * lr
                model.zero_grad()

In [271]:
model = nn.Sequential(nn.Linear(m,nh), nn.ReLU(), nn.Linear(nh,1))

In [272]:
fit()

tensor(11.5203, grad_fn=<L1LossBackward>)
tensor(11.0136, grad_fn=<L1LossBackward>)
tensor(10.6978, grad_fn=<L1LossBackward>)
tensor(10.3099, grad_fn=<L1LossBackward>)
tensor(9.9395, grad_fn=<L1LossBackward>)
tensor(9.4387, grad_fn=<L1LossBackward>)
tensor(9.1678, grad_fn=<L1LossBackward>)
tensor(8.5400, grad_fn=<L1LossBackward>)
tensor(8.0338, grad_fn=<L1LossBackward>)
tensor(7.4303, grad_fn=<L1LossBackward>)
tensor(6.7906, grad_fn=<L1LossBackward>)
tensor(5.7562, grad_fn=<L1LossBackward>)
tensor(5.1420, grad_fn=<L1LossBackward>)
tensor(4.0753, grad_fn=<L1LossBackward>)
tensor(3.1561, grad_fn=<L1LossBackward>)
tensor(2.3873, grad_fn=<L1LossBackward>)
tensor(1.7924, grad_fn=<L1LossBackward>)
tensor(1.6054, grad_fn=<L1LossBackward>)
tensor(1.5849, grad_fn=<L1LossBackward>)
tensor(1.6833, grad_fn=<L1LossBackward>)
tensor(1.5293, grad_fn=<L1LossBackward>)
tensor(1.5117, grad_fn=<L1LossBackward>)
tensor(1.3366, grad_fn=<L1LossBackward>)
tensor(1.2754, grad_fn=<L1LossBackward>)
tensor(1.397

In [ ]:
loss_func(model(xb), yb)